# TWITTER LINKS REPLACEMENT 

Ziel: ist es die links der Bilder, wenn sie nicht mehr funktionieren, automatsiche durch das selbe bild, aber einen alternativen link zu erstezen. 
Das sollte am besten in regelmäßigen Abständen gemacht werden. Basis des Datenstazes sind links von twitter-Bilder.

### FRAGEN: 

* In welcher Form / Datenbank werden die Bildinks abgespeichert ?
* Woher kommen die Fallback-links -> Müssen die noch automatisch generiert werden ? 
* Ich habe jetzt den Ordnern 10.Oktober und im subfolder complete verweendet, wo erreic ch di enicht gedownloaded BIlder ? 
* wie soll der Datenstaz erweitert werden 
* nach fallback links suchen und 

### Grobe Idee

1.  Datenbank fpr Bildlinks ist Voraustezung (Welche JSON, SQKLIte) => Notwenidg alternative Links oder Metdaten (Fallback-URLS)
    * Fallback URL = the backup URL, that is used when the traffc cannot reach the main page for some reason 
2. Überprüfung der Links 
    * Wichtig die Verfügbarkeit des Links zu Überpürfen (HTTP Statuscodes)
3. Atomatsiches Erstezten: 
    * wenn ein link nicht mehr funktonieren sollte, kann dieser ersetzt werden 
4. Notwenidg: Regelmäßigkeit in der Überprüfung = Shceduler benutzen 
5. Wenn keine fall-back links vorhanden, dann durch scraping oder twitter api nach alternativen für neue bilder suchen   

In [ ]:
import requests
import json
import time
from urllib.parse import urlparse

# Datenbank-Datei
DB_FILE = "image_links.json"

def load_links():
    """Lade Bildlinks aus der JSON-Datei."""
    try:
        with open(DB_FILE, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return []

def save_links(links):
    """Speichere Bildlinks in die JSON-Datei."""
    with open(DB_FILE, 'w') as file:
        json.dump(links, file, indent=4)

def check_link(url):
    """Überprüfe, ob ein Bildlink funktioniert."""
    try:
        response = requests.head(url, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def replace_broken_links(links):
    """Ersetze defekte Bildlinks durch alternative Links."""
    for entry in links:
        main_link = entry['main']
        alternatives = entry.get('alternatives', [])
        
        if not check_link(main_link):
            print(f"Link defekt: {main_link}")
            for alt in alternatives:
                if check_link(alt):
                    print(f"Ersetze durch: {alt}")
                    entry['main'] = alt
                    break
            else:
                print("Kein funktionierender alternativer Link gefunden.")
    return links

def main():
    """Hauptprogramm zur regelmäßigen Überprüfung."""
    links = load_links()
    links = replace_broken_links(links)
    save_links(links)
    print("Überprüfung abgeschlossen.")

if __name__ == "__main__":
    # Optional: Regelmäßige Überprüfung alle X Sekunden
    while True:
        main()
        time.sleep(3600)  # Überprüfung alle 3600 Sekunden (1 Stunde)


In [ ]:
[
    {
        "main": "https://twitter.com/example/image1.jpg",
        "alternatives": [
            "https://backup.com/image1.jpg",
            "https://anotherbackup.com/image1.jpg"
        ]
    },
    {
        "main": "https://twitter.com/example/image2.jpg",
        "alternatives": [
            "https://backup.com/image2.jpg"
        ]
    }
]


### Extensions: Integraton mit der Twitter API oder anderen Bildquellen 

Schritte:
1. Twitter API Zugang einrichten: 
    * Ich hab kein ENtwicler Konto und generiere API Schlüsse
    * Twitter API v2 für Bildabfragen 
2. Bild-Alternativen suchen:
    * Verwende die API, um Tweets mit ähnlichen Inhalten (z.b. Hashtags oder benutzerdefinierte Suchbegriffen zu durchsuchen )
3. Alternative Bildquellem:
    * Nutzen andre bildplattformen wie flickr oder unsplash (mit deren API )

In [ ]:
import tweepy
import logging

# Twitter API-Schlüssel
API_KEY = 'DEIN_API_KEY'
API_SECRET = 'DEIN_API_SECRET'
ACCESS_TOKEN = 'DEIN_ACCESS_TOKEN'
ACCESS_TOKEN_SECRET = 'DEIN_ACCESS_TOKEN_SECRET'

# Twitter API einrichten
def setup_twitter_api():
    auth = tweepy.OAuth1UserHandler(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    return tweepy.API(auth)

def search_alternative_images(api, query):
    """Suche alternative Bilder zu einem defekten Link."""
    try:
        tweets = api.search_tweets(q=query, count=10, tweet_mode='extended')
        alternatives = []
        for tweet in tweets:
            media = tweet.entities.get('media', [])
            for item in media:
                if item['type'] == 'photo':
                    alternatives.append(item['media_url_https'])
        return alternatives
    except Exception as e:
        logging.error(f"Fehler bei der Twitter-Suche: {e}")
        return []

# Beispielverwendung
if __name__ == "__main__":
    api = setup_twitter_api()
    broken_link = "https://twitter.com/example/image1.jpg"
    query = "relevant hashtag or keywords"
    alternatives = search_alternative_images(api, query)
    print("Gefundene Alternativen:", alternatives)


In [ ]:
import logging

# Logging konfigurieren
logging.basicConfig(
    filename='image_link_checker.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def log_broken_link(link):
    """Logge einen defekten Link."""
    logging.warning(f"Defekter Link gefunden: {link}")

def log_replacement(old_link, new_link):
    """Logge die Ersetzung eines defekten Links."""
    logging.info(f"Link ersetzt: {old_link} -> {new_link}")

def log_error(error_message):
    """Logge Fehler."""
    logging.error(f"Fehler: {error_message}")

# Beispielverwendung
if __name__ == "__main__":
    log_broken_link("https://twitter.com/example/image1.jpg")
    log_replacement("https://twitter.com/example/image1.jpg", "https://backup.com/image1.jpg")
    log_error("Keine alternativen Links gefunden.")
